In [24]:
import json
from transformers import BertTokenizerFast
import uuid

In [3]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [29]:
label_map = {
    'B-1': 0,
    'I-1': 1,
    'B-2': 2,
    'I-2': 3,
    'B-3': 4,
    'I-3': 5,
    'B-4': 6,
    'I-4': 7,
}

result = {'label_map': label_map, 'examples': []}
with open('data/hierarchy.json') as rf:
    for ind, l in enumerate(rf.readlines()):
        l = l.strip()
        obj = json.loads(l)
        text = obj['text']
        labels = []
        curr_end = 0
        sorted_annotations = sorted(obj['annotations'], key=lambda x: x['start_offset'])
        for annotation in sorted_annotations:
            label = annotation['label']
            start_offset = annotation['start_offset']
            end_offset = annotation['end_offset']
            if end_offset <= curr_end:
                print(obj['annotations'])
                raise Exception('Something is wrong with the ordering of the offsets')
            curr_end = end_offset
            tokens = tokenizer.encode(text[start_offset:end_offset])
            seq_tokens = tokens[1:-1]
            for i in range(len(seq_tokens)):
                if i == 0:
                    tok_label = f'B-{label}'
                    labels.append(tok_label)
                else:
                    tok_label = f'I-{label}'
                    labels.append(tok_label)
        tokens = tokenizer.encode(text)
        assert len(tokens)-2 == len(labels)
        assert len(tokens) < 513
        result['examples'].append({'text': text, 'labels': labels})
with open('data/labels.json', 'w') as wf:
    json.dump(result, wf)